In [2]:
%load_ext autoreload
%autoreload 2
import os
import sys
import time
import tensorflow as tf
sys.path.append("../")

from pgn import PGN
from loss import coverage_loss, mask_coverage_loss
from batcher import batcher
from utils.config import CKPT_DIR
from utils.saveLoader import Vocab
from utils.params import get_default_params
from utils.config_gpu import config_gpu
config_gpu()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1 Physical GPUs, 1 Logical GPUs


In [3]:
%run ../utils/train.py --batch_size 4 --test_run True

1 Physical GPUs, 1 Logical GPUs
Building vocab ...
Building the model ...
Creating the checkpoint manager
Initializing from scratch.
Starting the training ...
Epoch 1 Batch 1 Loss 3.4151 norm_loss 3.0928 cover_loss 0.3224
Epoch 1 Batch 2 Loss 5.9242 norm_loss 5.3650 cover_loss 0.5592
Epoch 1 Batch 3 Loss 6.8290 norm_loss 6.1843 cover_loss 0.6447
Epoch 1 Batch 4 Loss 4.4605 norm_loss 4.0395 cover_loss 0.4211
Epoch 1 Batch 5 Loss 6.4113 norm_loss 5.8060 cover_loss 0.6053
Epoch 1 Batch 6 Loss 4.6694 norm_loss 4.2286 cover_loss 0.4408
Epoch 1 Batch 7 Loss 5.5075 norm_loss 4.9878 cover_loss 0.5197
Epoch 1 Batch 8 Loss 5.5762 norm_loss 5.0499 cover_loss 0.5263
Epoch 1 Batch 9 Loss 6.6233 norm_loss 5.9983 cover_loss 0.6250
Epoch 1 Batch 10 Loss 4.4605 norm_loss 4.0395 cover_loss 0.4211
Epoch 1 Batch 11 Loss 3.3450 norm_loss 3.0292 cover_loss 0.3158
my test break
共花了 80.83 s


In [2]:
params = get_default_params()
model = PGN(params)
vocab = Vocab(params["vocab_path"], params["vocab_size"])

In [3]:
# def loss_function(real, pred, padding_mask):
#     loss = 0
#     for t in range(real.shape[1]):
#         # if padding_mask:
#         try:
#             # real[:,t] shape (batch_size, ) (32, ) 表示这批样本的第t个词
#             # pred[:, t, :] shape （batch_size, vocab_size) (32, 32252) 表示这批样本，第t个词的概率分布
#             loss_ = loss_object(real[:, t], pred[:, t, :])
#             mask = tf.cast(padding_mask[:, t], dtype=loss_.dtype)
#             loss_ *= mask
#             loss_ = tf.reduce_mean(loss_, axis=0)  # batch-wise
#             loss += loss_
#         # else:
#         except:
#             loss_ = loss_object(real[:, t], pred[:, t, :])
#             loss_ = tf.reduce_mean(loss_, axis=0)  # batch-wise
#             loss += loss_
#     return tf.reduce_mean(loss)

In [4]:
dataset = batcher(vocab, params)
encoder_batch_data, decoder_batch_data = next(iter(dataset))

In [5]:
enc_inp = encoder_batch_data["enc_input"]
extended_enc_input = encoder_batch_data["extended_enc_input"]
max_oov_len = encoder_batch_data["max_oov_len"]
dec_input = decoder_batch_data["dec_input"]
dec_target = decoder_batch_data["dec_target"]
cov_loss_wt=1
enc_pad_mask=encoder_batch_data["sample_encoder_pad_mask"]
padding_mask=decoder_batch_data["sample_decoder_pad_mask"]

In [6]:
enc_output, enc_hidden = model.call_encoder(enc_inp)
dec_hidden = enc_hidden

In [7]:
predictions, _, attentions, coverages = model(dec_input,
                                              dec_hidden,
                                              enc_output,
                                              dec_target,
                                              extended_enc_input,
                                              max_oov_len,
                                              enc_pad_mask=enc_pad_mask,
                                              use_coverage=True,
                                              prev_coverage=None)

In [8]:
# 定义损失函数
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
def loss_function(real, pred, padding_mask):
    # <PAD> 和 <UNK> 的损失都不算
    loss_ = loss_object(real, pred)

    mask = tf.cast(padding_mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [9]:
from loss import coverage_loss
loss_function(dec_target, predictions, padding_mask)

<tf.Tensor: id=14123, shape=(), dtype=float32, numpy=5.425539>

In [10]:
cl = cov_loss_wt * coverage_loss(attentions, coverages, padding_mask)
cl

<tf.Tensor: id=14341, shape=(), dtype=float32, numpy=0.9807154>

进入coverage_loss

In [13]:
attn_dists = attentions
pad_mask=False

# shape (batch_size, max_len_x). Initial coverage is zero.
coverage = tf.zeros_like(attn_dists[0])
# Coverage loss per decoder timestep. Will be list length max_dec_steps containing shape (batch_size).
cover_losses = []
for t, a in enumerate(attn_dists):
    cover_loss = tf.reduce_sum(tf.minimum(a, coverage), [1])  # calculate the coverage loss for this step
    if pad_mask:
        mask = tf.cast(padding_mask[:,t], dtype=cover_loss.dtype)
        cover_losses.append(cover_loss * mask)
    else:
        cover_losses.append(cover_loss)
    # cover_losses.append(cover_loss)
    # update the coverage vector
    # batch size ,max_x_len
    coverage += a
loss = tf.reduce_mean(cover_losses)
loss

<tf.Tensor: id=15089, shape=(), dtype=float32, numpy=0.9807154>

In [34]:
cover_losses = []

In [46]:
cover,attn = coverages[0], attn_dists[0]

In [48]:
cover = tf.squeeze(cover)
# attn =  tf.squeeze(attn)

In [49]:
cover.shape,attn.shape

(TensorShape([64, 460]), TensorShape([64, 460]))

In [58]:
tf.reduce_sum(tf.minimum(cover, attn), axis=-1)

<tf.Tensor: id=15129, shape=(64,), dtype=float32, numpy=
array([1.0000001 , 0.99999976, 1.        , 0.9999999 , 1.0000002 ,
       1.0000001 , 1.0000001 , 0.99999994, 0.99999994, 1.        ,
       1.        , 0.99999976, 1.0000001 , 1.0000001 , 0.99999994,
       1.        , 0.9999998 , 1.0000001 , 0.9999999 , 0.9999999 ,
       0.99999976, 1.0000002 , 1.        , 0.9999999 , 0.9999999 ,
       1.0000001 , 1.        , 0.9999999 , 1.        , 1.        ,
       1.        , 1.        , 1.0000001 , 0.9999998 , 0.9999999 ,
       0.99999994, 0.9999999 , 0.99999976, 1.        , 0.9999998 ,
       1.0000001 , 0.9999999 , 0.9999997 , 1.        , 0.99999976,
       1.        , 0.99999994, 0.9999999 , 1.        , 0.99999994,
       1.        , 0.9999999 , 0.99999976, 1.0000001 , 1.        ,
       1.        , 1.        , 0.99999964, 1.        , 1.        ,
       1.        , 1.        , 0.9999998 , 0.99999994], dtype=float32)>

In [101]:
from loss import coverage_loss
coverage_loss(attn_dists, coverages, padding_mask)

<tf.Tensor: id=17488, shape=(), dtype=float32, numpy=0.52283657>

In [94]:
cover_losses = []
for cover, attn in zip(coverages, attn_dists):
    # cover, atten shape (batch_size, enc_len)
    cover = tf.squeeze(cover)
    # cover_loss_ shape (batch_size)
    cover_loss_ = tf.reduce_sum(tf.minimum(cover, attn), axis=-1)
    cover_losses.append(cover_loss_)
    
cover_losses = tf.stack(cover_losses, 1)
mask = tf.cast(padding_mask, dtype=cover_loss_.dtype)
cover_losses *= mask
loss = tf.reduce_mean(cover_losses)

    

In [90]:
tf.reduce_mean(cover_losses)

<tf.Tensor: id=16405, shape=(), dtype=float32, numpy=0.52283657>

In [85]:
loss = tf.reduce_sum(tf.reduce_mean(cover_losses, axis=0))
loss

<tf.Tensor: id=16399, shape=(), dtype=float32, numpy=27.1875>

配置保存点管理器

In [4]:
# checkpoint = tf.train.Checkpoint(Seq2Seq=model)
# checkpoint_manager = tf.train.CheckpointManager(checkpoint, CKPT_DIR, max_to_keep=5)
# checkpoint.restore(checkpoint_manager.latest_checkpoint)
# if checkpoint_manager.latest_checkpoint:
#     print("Restored from {}".format(checkpoint_manager.latest_checkpoint))
# else:
#     print("Initializing from scratch.")

Restored from E:\GitHub\QA-abstract-and-reasoning\data\checkpoints\training_checkpoints\test_model-1


In [11]:
# from utils.train_helper import train_model
# train_model(model, vocab, params, checkpoint_manager)

In [7]:
params['max_train_steps']

20736

In [9]:
checkpoint_prefix = os.path.join(CKPT_DIR, "test_model")
checkpoint.save(file_prefix=checkpoint_prefix)

'E:\\GitHub\\QA-abstract-and-reasoning\\data\\checkpoints\\training_checkpoints\\test_model-1'

## 一探train_model究竟

- 摒弃`@tf.funtion`

In [4]:
epochs = params['epochs']
optimizer = tf.keras.optimizers.Adagrad(params['learning_rate'],
                                            initial_accumulator_value=params['adagrad_init_acc'],
                                            clipnorm=params['max_grad_norm'])
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
# optimizer.get_config(), loss_object.get_config()

In [5]:
def loss_function(real, pred, padding_mask):
    loss = 0
    for t in range(real.shape[1]):
        # if padding_mask:
        try:
            loss_ = loss_object(real[:, t], pred[:, t, :])
            mask = tf.cast(padding_mask[:, t], dtype=loss_.dtype)
            loss_ *= mask
            loss_ = tf.reduce_mean(loss_, axis=0)  # batch-wise
            loss += loss_
        # else:
        except:
            loss_ = loss_object(real[:, t], pred[:, t, :])
            loss_ = tf.reduce_mean(loss_, axis=0)  # batch-wise
            loss += loss_
    return tf.reduce_mean(loss)

In [35]:
# @tf.funtion
def train_step(enc_inp, extended_enc_input, max_oov_len,
               dec_input, dec_target, cov_loss_wt,
               enc_pad_mask, padding_mask=None):
    batch_loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = model.call_encoder(enc_inp)
        # 第一个隐藏层输入
        dec_hidden = enc_hidden
        # 逐个预测序列
        predictions, _, attentions, coverages = model(dec_input,
                                                      dec_hidden,
                                                      enc_output,
                                                      dec_target,
                                                      extended_enc_input,
                                                      max_oov_len,
                                                      enc_pad_mask=enc_pad_mask,
                                                      use_coverage=True,
                                                      prev_coverage=None)
    
        batch_loss = loss_function(dec_target, predictions, padding_mask) + \
                     cov_loss_wt * coverage_loss(attentions, coverages, padding_mask)
        
#     variables = model.encoder.trainable_variables + model.decoder.trainable_variables + \
#                 model.attention.trainable_variables + model.pointer.trainable_variables
    variables = model.trainable_variables
    gradients = tape.gradient(batch_loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [36]:
start = time.time()
dataset = batcher(vocab, params)
total_loss = 0
step = 0

In [37]:
# 进入for 循环
# for encoder_batch_data, decoder_batch_data in dataset:
encoder_batch_data, decoder_batch_data = next(iter(dataset))

In [38]:
batch_loss = train_step(enc_inp = encoder_batch_data["enc_input"],
                        extended_enc_input = encoder_batch_data["extended_enc_input"],
                        max_oov_len = encoder_batch_data["max_oov_len"],
                        dec_input = decoder_batch_data["dec_input"],
                        dec_target = decoder_batch_data["dec_target"],
                        cov_loss_wt=0.5,
                        enc_pad_mask=encoder_batch_data["sample_encoder_pad_mask"],
                        padding_mask=decoder_batch_data["sample_decoder_pad_mask"])

进入train_step